# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 9 2022 8:30AM,245999,10,bps200075,Beach Patient Assistance,Released
1,Sep 9 2022 8:18AM,246017,10,0085969997,ISDIN Corporation,Released
2,Sep 9 2022 8:18AM,246017,10,0085970000,ISDIN Corporation,Released
3,Sep 9 2022 8:18AM,246017,10,0085970003,ISDIN Corporation,Released
4,Sep 9 2022 8:18AM,246017,10,0085970012,ISDIN Corporation,Released
5,Sep 9 2022 8:18AM,246017,10,0085970023,ISDIN Corporation,Released
6,Sep 9 2022 8:18AM,246017,10,0085970064,ISDIN Corporation,Released
7,Sep 9 2022 8:18AM,246017,10,0085970068,ISDIN Corporation,Released
8,Sep 9 2022 8:18AM,246017,10,0085970119,ISDIN Corporation,Released
9,Sep 9 2022 8:18AM,246017,10,0085970120,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,246013,Released,3
31,246014,Released,1
32,246015,Released,1
33,246016,Released,1
34,246017,Released,13


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
246013,NaN,NaN,NaN,3.0
246014,NaN,NaN,NaN,1.0
246015,NaN,NaN,NaN,1.0
246016,NaN,NaN,NaN,1.0
246017,NaN,NaN,NaN,13.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
245651,0.0,0.0,0.0,1.0
245733,0.0,0.0,0.0,1.0
245744,0.0,0.0,1.0,0.0
245758,1.0,0.0,0.0,0.0
245763,0.0,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
245651,0.0,0,0,1
245733,0.0,0,0,1
245744,0.0,0,1,0
245758,1.0,0,0,0
245763,0.0,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,245651,0.0,0,0,1
1,245733,0.0,0,0,1
2,245744,0.0,0,1,0
3,245758,1.0,0,0,0
4,245763,0.0,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,245651,0.0,,,1
1,245733,0.0,,,1
2,245744,0.0,,1,
3,245758,1.0,,,
4,245763,0.0,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 9 2022 8:30AM,245999,10,Beach Patient Assistance
1,Sep 9 2022 8:18AM,246017,10,ISDIN Corporation
14,Sep 9 2022 8:09AM,246016,10,"Amcyte Pharma, Inc."
15,Sep 9 2022 8:08AM,246015,10,MedStone Pharma LLC
16,Sep 9 2022 8:07AM,246014,10,Beach Products Inc
17,Sep 9 2022 8:04AM,246013,10,Emerginnova
20,Sep 9 2022 7:57AM,246012,19,"AdvaGen Pharma, Ltd"
30,Sep 8 2022 5:01PM,246007,21,"NBTY Global, Inc."
31,Sep 8 2022 4:59PM,245651,21,"NBTY Global, Inc."
32,Sep 8 2022 4:58PM,245774,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Sep 9 2022 8:30AM,245999,10,Beach Patient Assistance,0.0,,,1
1,Sep 9 2022 8:18AM,246017,10,ISDIN Corporation,0.0,,,13
2,Sep 9 2022 8:09AM,246016,10,"Amcyte Pharma, Inc.",0.0,,,1
3,Sep 9 2022 8:08AM,246015,10,MedStone Pharma LLC,0.0,,,1
4,Sep 9 2022 8:07AM,246014,10,Beach Products Inc,0.0,,,1
5,Sep 9 2022 8:04AM,246013,10,Emerginnova,0.0,,,3
6,Sep 9 2022 7:57AM,246012,19,"AdvaGen Pharma, Ltd",0.0,,,10
7,Sep 8 2022 5:01PM,246007,21,"NBTY Global, Inc.",0.0,,,1
8,Sep 8 2022 4:59PM,245651,21,"NBTY Global, Inc.",0.0,,,1
9,Sep 8 2022 4:58PM,245774,21,"NBTY Global, Inc.",0.0,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 9 2022 8:30AM,245999,10,Beach Patient Assistance,1,,
1,Sep 9 2022 8:18AM,246017,10,ISDIN Corporation,13,,
2,Sep 9 2022 8:09AM,246016,10,"Amcyte Pharma, Inc.",1,,
3,Sep 9 2022 8:08AM,246015,10,MedStone Pharma LLC,1,,
4,Sep 9 2022 8:07AM,246014,10,Beach Products Inc,1,,
5,Sep 9 2022 8:04AM,246013,10,Emerginnova,3,,
6,Sep 9 2022 7:57AM,246012,19,"AdvaGen Pharma, Ltd",10,,
7,Sep 8 2022 5:01PM,246007,21,"NBTY Global, Inc.",1,,
8,Sep 8 2022 4:59PM,245651,21,"NBTY Global, Inc.",1,,
9,Sep 8 2022 4:58PM,245774,21,"NBTY Global, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 9 2022 8:30AM,245999,10,Beach Patient Assistance,1,,
1,Sep 9 2022 8:18AM,246017,10,ISDIN Corporation,13,,
2,Sep 9 2022 8:09AM,246016,10,"Amcyte Pharma, Inc.",1,,
3,Sep 9 2022 8:08AM,246015,10,MedStone Pharma LLC,1,,
4,Sep 9 2022 8:07AM,246014,10,Beach Products Inc,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 9 2022 8:30AM,245999,10,Beach Patient Assistance,1.0,NaN,NaN
1,Sep 9 2022 8:18AM,246017,10,ISDIN Corporation,13.0,NaN,NaN
2,Sep 9 2022 8:09AM,246016,10,"Amcyte Pharma, Inc.",1.0,NaN,NaN
3,Sep 9 2022 8:08AM,246015,10,MedStone Pharma LLC,1.0,NaN,NaN
4,Sep 9 2022 8:07AM,246014,10,Beach Products Inc,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 9 2022 8:30AM,245999,10,Beach Patient Assistance,1.0,0.0,0.0
1,Sep 9 2022 8:18AM,246017,10,ISDIN Corporation,13.0,0.0,0.0
2,Sep 9 2022 8:09AM,246016,10,"Amcyte Pharma, Inc.",1.0,0.0,0.0
3,Sep 9 2022 8:08AM,246015,10,MedStone Pharma LLC,1.0,0.0,0.0
4,Sep 9 2022 8:07AM,246014,10,Beach Products Inc,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2705650,50.0,3.0,4.0
15,737918,29.0,9.0,75.0
16,737978,9.0,0.0,0.0
18,245934,0.0,1.0,0.0
19,492018,12.0,0.0,0.0
20,246003,5.0,8.0,0.0
21,1720443,6.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2705650,50.0,3.0,4.0
1,15,737918,29.0,9.0,75.0
2,16,737978,9.0,0.0,0.0
3,18,245934,0.0,1.0,0.0
4,19,492018,12.0,0.0,0.0
5,20,246003,5.0,8.0,0.0
6,21,1720443,6.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,50.0,3.0,4.0
1,15,29.0,9.0,75.0
2,16,9.0,0.0,0.0
3,18,0.0,1.0,0.0
4,19,12.0,0.0,0.0
5,20,5.0,8.0,0.0
6,21,6.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,50.0
1,15,Released,29.0
2,16,Released,9.0
3,18,Released,0.0
4,19,Released,12.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21
Status,,,,,,,
Completed,4.0,75.0,0.0,0.0,0.0,0.0,0.0
Executing,3.0,9.0,0.0,1.0,0.0,8.0,1.0
Released,50.0,29.0,9.0,0.0,12.0,5.0,6.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21
0,Completed,4.0,75.0,0.0,0.0,0.0,0.0,0.0
1,Executing,3.0,9.0,0.0,1.0,0.0,8.0,1.0
2,Released,50.0,29.0,9.0,0.0,12.0,5.0,6.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21
0,Completed,4.0,75.0,0.0,0.0,0.0,0.0,0.0
1,Executing,3.0,9.0,0.0,1.0,0.0,8.0,1.0
2,Released,50.0,29.0,9.0,0.0,12.0,5.0,6.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()